In [ ]:
BRANCH = 'main'

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]


In [ ]:
# If you're not using Colab, you might need to upgrade jupyter notebook to avoid the following error:
# 'ImportError: IProgress not found. Please update jupyter and ipywidgets.'

! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

# Please restart the kernel after running this cell

In [ ]:
from nemo.utils.exp_manager import exp_manager
from nemo.collections import nlp as nemo_nlp

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

# Task Description
Automatic Speech Recognition (ASR) systems typically generate text with no punctuation and capitalization of the words. 
This tutorial explains how to implement a model in NeMo that will predict punctuation and capitalization for each word in a sentence to make ASR output more readable and to boost performance of the named entity recognition, machine translation or text-to-speech models. 
We'll show how to train a model for this task using a pre-trained BERT model. 
For every word in our training dataset we’re going to predict:

- punctuation mark that should follow the word and
- whether the word should be capitalized

# Dataset
This model can work with any dataset as long as it follows the format specified below. 
The training and evaluation data is divided into *2 files: text.txt and labels.txt*. 
Each line of the **text.txt** file contains text sequences, where words are separated with spaces: [WORD] [SPACE] [WORD] [SPACE] [WORD], for example:



```
when is the next flight to new york
the next flight is ...
...
```



The **labels.txt** file contains corresponding labels for each word in text.txt, the labels are separated with spaces. Each label in labels.txt file consists of 2 symbols:

- the first symbol of the label indicates what punctuation mark should follow the word (where O means no punctuation needed);
- the second symbol determines if a word needs to be capitalized or not (where U indicates that the word should be upper cased, and O - no capitalization needed.)

In this tutorial, we are considering only commas, periods, and question marks the rest punctuation marks were removed. To use more punctuation marks, update the dataset to include desired labels, no changes to the model needed. 

Each line of the **labels.txt** should follow the format: 
[LABEL] [SPACE] [LABEL] [SPACE] [LABEL] (for labels.txt). 
For example, labels for the above text.txt file should be:



```
OU OO OO OO OO OO OU ?U
OU OO OO OO ...
...
```



The complete list of all possible labels for this task used in this tutorial is: `OO, ,O, .O, ?O, OU, ,U, .U, ?U.`

## Download and preprocess the data¶

In this notebook we are going to use a subset of English examples from the [Tatoeba collection of sentences](https://tatoeba.org/eng) this script will download and preprocess the Tatoeba data [NeMo/examples/nlp/token_classification/get_tatoeba_data.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/data/get_tatoeba_data.py). Note, for further experiments with the model, set NUM_SAMPLES=-1 and consider including other datasets to improve model performance. 


In [ ]:
DATA_DIR = "PATH_TO_DATA"
WORK_DIR = "PATH_TO_CHECKPOINTS_AND_LOGS"
MODEL_CONFIG = "punctuation_capitalization_config.yaml"

# model parameters
BATCH_SIZE = 128
MAX_SEQ_LENGTH = 64
LEARNING_RATE = 0.00002
NUM_SAMPLES = 10000

In [ ]:
## download get_tatoeba_data.py script to download and preprocess the Tatoeba data
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(WORK_DIR + '/get_tatoeba_data.py'):
    print('Downloading get_tatoeba_data.py...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/data/get_tatoeba_data.py', WORK_DIR)
else:
    print ('get_tatoeba_data.py is already exists')

In [ ]:
# download and preprocess the data
# --clean_dir flag deletes raw Tataoeba data, remove the flag to avoid multiple data downloads if you want to experiment with the dataset size
! python $WORK_DIR/get_tatoeba_data.py --data_dir $DATA_DIR --num_sample $NUM_SAMPLES --clean_dir

after execution of the above cell, your data folder will contain the following 4 files needed for training (raw Tatoeba data could be present if `--clean_dir` was not used):
- labels_dev.txt
- labels_train.txt
- text_dev.txt
- text_train.txt


In [ ]:
! ls -l {DATA_DIR}

In [ ]:
# let's take a look at the data 
print('Text:')
! head -n 5 {DATA_DIR}/text_train.txt

print('\nLabels:')
! head -n 5 {DATA_DIR}/labels_train.txt

# Model Configuration

In the Punctuation and Capitalization Model, we are jointly training two token-level classifiers on top of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model: 
- one classifier to predict punctuation and
- the other one - capitalization.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, token classifiers, optimizer and schedulers, dataset and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

In [ ]:
# download the model's configuration file 
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file is already exists')

In [ ]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

# Setting up Data within the config

Among other things, the config file contains dictionaries called dataset, train_ds and validation_ds. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

If both training and evaluation files are located in the same directory, simply specify `model.dataset.data_dir`, like we are going to do below.
However, if your evaluation files are located in a different directory, or you want to use multiple datasets for evaluation, specify paths to the directory(ies) with evaluation file(s) in the following way:

`model.validation_ds.ds_item=[PATH_TO_DEV1,PATH_TO_DEV2]` (Note no space between the paths and square brackets).

Also notice that some configs, including `model.dataset.data_dir`, have `???` in place of paths, this values are required to be specified by the user.

Let's now add the data directory path to the config.

In [ ]:
# in this tutorial train and dev data is located in the same folder, so it is enought to add the path of the data directory to our config
config.model.dataset.data_dir = DATA_DIR

# Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem!

Let's first instantiate a Trainer object!

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# For mixed precision training, use precision=16 and amp_level=O1

# Reduces maximum number of epochs to 1 for a quick training
config.trainer.max_epochs = 1

# Remove distributed training flags
config.trainer.accelerator = None

trainer = pl.Trainer(**config.trainer)

# Setting up a NeMo Experiment¶

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it!

In [ ]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

# Model Training

Before initializing the model, we might want to modify some of the model configs. For example, we might want to modify the pretrained BERT model.

In [ ]:
# complete list of supported BERT-like models
nemo_nlp.modules.get_pretrained_lm_models_list()

PRETRAINED_BERT_MODEL = "bert-base-uncased"

In [ ]:
# add the specified above model parameters to the config
config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.train_ds.batch_size = BATCH_SIZE
config.model.validation_ds.batch_size = BATCH_SIZE
config.model.optim.lr = LEARNING_RATE
config.model.train_ds.num_samples = NUM_SAMPLES
config.model.validation_ds.num_samples = NUM_SAMPLES


Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders we'll be prepared for training and evaluation.
Also, the pretrained BERT model will be downloaded, note it can take up to a few minutes depending on the size of the chosen BERT model.

In [ ]:
# initialize the model
# during this stage, the dataset and data loaders we'll be prepared for training and evaluation
config.trainer.max_epochs = 3
model = nemo_nlp.models.PunctuationCapitalizationModel(cfg=config.model, trainer=trainer)

## Monitoring training progress
Optionally, you can create a Tensorboard visualization to monitor training progress.

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir {exp_dir}
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
# start the training
trainer.fit(model)

# Inference

To see how the model performs, let’s run inference on a few examples.

In [ ]:
# define the list of queiries for inference
queries = [
        'we bought four shirts and one mug from the nvidia gear store in santa clara',
        'what can i do for you today',
        'how are you',
        'how is the weather in',
    ]
inference_results = model.add_punctuation_capitalization(queries)
print()

for query, result in zip(queries, inference_results):
    print(f'Query   : {query}')
    print(f'Combined: {result.strip()}\n')

Inference for large amounts of text can be performed by a script [examples/nlp/token_classification/punctuate_capitalize_infer.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/punctuate_capitalize_infer.py>)

```
python punctuate_capitalize_infer.py \
    --input_manifest <PATH_TO_INPUT_MANIFEST> \
    --output_manifest <PATH_TO_OUTPUT_MANIFEST> \
    --pretrained_name punctuation_en_bert \
    --max_seq_length 64 \
    --margin 16 \
    --step 8
```

`<PATH_TO_INPUT_MANIFEST>` is a path to NeMo [ASR manifest](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/asr/datasets.html) which contains text in which you need to restore punctuation and capitalization. If manifest contains `'pred_text'` key, then `'pred_text'` elements will be processed. Otherwise, punctuation and capitalization will be restored in `'text'` elements.

`<PATH_TO_OUTPUT_MANIFEST>` is a path to NeMo ASR manifest into which result will be saved. The text with restored
punctuation and capitalization is saved into `'pred_text'` elements if `'pred_text'` key is present in
input manifest. Otherwise result will be saved into `'text'` elements.

Alternatively you can pass text for restoring punctuation and capitalization as plain text. See help for parameters `--input_text` and `--output_text` of the script
[punctuate_capitalize_infer.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/punctuate_capitalize_infer.py>).

The script `examples/nlp/token_classification/punctuate_capitalize_infer.py` can restore punctuation and capitalization in a text of arbitrary length. Long sequences are split into segments
`--max_seq_length - 2` tokens each. Each segment starts and ends with `[CLS]` and `[SEP]`
tokens correspondingly. Every segment is offset to the previous one by `--step` tokens. For example, if
every character is a token, `--max_seq_length=5`, `--step=2`, then text `"hello"` will be split into
segments `[['[CLS]', 'h', 'e', 'l', '[SEP]'], ['[CLS]', 'l', 'l', 'o', '[SEP]']]`.

If segments overlap, then predicted probabilities for a token present in several segments are multiplied before selecting the best candidate.

Splitting leads to pour performance of a model near edges of segments. Use parameter `--margin` to discard `--margin`
probabilities predicted for `--margin` tokens near segment edges. For example, if every character is a token, `--max_seq_length=5`, `--step=1`, `--margin=1`, then text `"hello"` will be split into segments `[['[CLS]', 'h', 'e', 'l', '[SEP]'], ['[CLS]', 'e', 'l', 'l', '[SEP]'], ['[CLS]', 'l', 'l', 'o', '[SEP]']]`. Before calculating actual predictions, probabilities for tokens marked by asterisk are removed: `[['[CLS]', 'h', 'e', 'l'*, '[SEP]'*], ['[CLS]'*, 'e'*, 'l', 'l'*, '[SEP]'*], ['[CLS]'*, 'l'*, 'l', 'o', '[SEP]']]`.

You may use parameters `max_seq_length`, `step`, and `margin` in `add_punctuation_capitalization` method.

The text in the following example is from IWSLT 2019 test dataset.

In [ ]:
inference_results = model.add_punctuation_capitalization(
    [
        "each of these songs represents a scene a movement in some cases a sonic revolution that completely altered the "
        "course of popular music they're all also calling cards almost for those cities songs totally linked with their "
        "city's identity and might be why you probably consider them to be music cities now the magical mythical thing "
        "the thing we kind of all love about stories like these is that those cities weren't doing anything in particular "
        "to make those moments happen there's no formula for capturing lightning in a bottle a formula didn't give us "
        "grunge music or introduce tupock to dr dray and there's definitely no blueprint for how to open your record "
        "business in a south memphis neighborhood that turns out is home to booker t jones william bell and albert king "
        "so this is just something that happens and right when the stars perfectly align great music just happens and "
        "in the meantime new york and nashville can churn out the hits that come through our radios define our "
        "generations and soundtrack our weddings and our funerals and everything in between i don't know about you but "
        "the very idea of that is just deadly boring to me there are musicians all around you making powerful important "
        "music and thanks to the internet and it's limitless possibilities for creators to create music and fans to "
        "discover that music those zyite guy songs don't have to be handed down to us from some conference room full of "
        "songwriters in a corporate high rise but also and more importantly we can't decide that it's just something that "
        "happens because music is about so much more than hits those big iconic moments that change everything it's more "
        "than just entertainment for so many of us music is truly a way to navigate life a means of self expression sure "
        "but it also helps us find ourrselfel worse and figure out who we are it connects us with other people as almost "
        "nothing else can across language barriers across social and cultural and economic divides music makes us smarter "
        "and healthier and happier music is necessary what if you lived in a city that believed that that said we're not "
        "waiting for that hit song to define us we're a music city because music is necessary by seeing music as "
        "necessary a city can build two things 1st an ecosystem to support the development of professional musicians "
        "and music business and 2nd a receptive and engaged audience to sustain them and those are the two critical "
        "elements of a music city a city whose leaders recognize the importance of music for our development as "
        "individuals our connection as a community and our viability as a vibrant place to live see smart cities music "
        "cities know that thriving night lifef a creative class culture is what attracts young talented people to cities "
        "it's what brings that lightning and no we can't predict the next egg that will hatch but we can create a city "
        "that acts like an incubator to do that 1st we got to know what we've got that means identifying and quantifying "
        "our assets we need to know them backward and forward from who and what and where they are to what their impact "
        "is on the economy let's count our recording studios and our record labels our historic landmarks and our "
        "hardcore punk clubs we should count monthly free jazz nights and weekly folk jams music schools artist "
        "development instrument shops every lay than every luther music museums open once a open year round and music "
        "festivals open just one weekend year now ideally through this process we'll create an actual asset map "
        "dropping a pin for each one allowing us to see exactly what we've got and where organic momentum is already "
        "happening because it's not enough to paint in broad strokes here when it comes to specific support for music "
        "locally and a broad understanding of a music brand nationally you've got to have the receipts next we'll need "
        "to identify our challenges now it's important to knowe that for the most part this won't be just the opposite "
        "of step one we won't gain a whole lot by simply thinking about what's missing from our map instead we need to "
        "approach this more holistically there are lots of music venues on our map awesome but are they struggling do "
        "we have a venue ladder which just means can an artist starting out at a coffee house open mike see a clear path "
        "for how they'll grow from that 25 seat room to a hundred seat room and so on or are we expecting them to go from "
        "a coffee house to a colossum maybe our challenges lie in city infrastructure public transportation affordable "
        "housing maybe like in london where the number of music venues went from 400 in 2010 to 100 in 2015 we need to "
        "think about protections against gentrification the mayor of london in december of last year actually added "
        "something called the agent of change principle to the city's comprehensive plan the name says it all if a real "
        "estate developer wants to build condos next to an existing music venue the developer is the agent of change they "
        "have to take the necessary steps for noise mitigation next and this is a very big one we need leadership and we "
        "need a strategy now we know there's a lot of magic in this mix a lot of right people right place right time and "
        "that will never stop being an important element of the way music is made the way some of the best most enduring "
        "music is made but there cannot be a leadership vacuum in 2018 surriving music cities don't often happen and "
        "don't have to happen accidentally we need elected officials who recognize the power of music and elevate the "
        "voices of creatives and they're ready to put a strategy in place in music cities from berlin to paris to bogata "
        "music advisory councils ensure that musicians have a seat at the table they're volunteer councils and they work "
        "directly with a designated advocate inside of city hall or even the chamber of commerce the strongest strategies "
        "will build music community supports like this one inward while also exporting music outward they go hand in hand "
        "when we look inward we create that place that musicians want to live and when we look outward we build "
        "opportunities for them to advance their career while also driving attention back to our city and leveraging music "
        "as a talent attraction tool and here's something else that will help with that we've got to figure out who we are "
        "now when i say austin you probably think live music capital and why because in 1991 leadership in austin saw "
        "something percolating with an existing asset and they chose to own it by recognizing that momentum naming it and "
        "claiming it they inevitably caused more live music venues to open existing spaces to add live music to their "
        "repertoire and they created a swell of civic buy in around the idea which meant that it wasn't just a slogan in "
        "some tourism pamphlet was something that locals really started to believe and take pride in now generally "
        "speaking what austin created is just an assets based narrative and when we think back to step one we know that "
        "every city will not tick every box many cities won't have recording studios like memphis or a songwriter and "
        "publishing scene like nashville and that's not a deal breaker we simply have to find the momentum happening in "
        "our city what are our unique assets in comparison to no other place so if all of that sounds like something "
        "you'd like to happen where you live here are three things you can do to move the needle 1st you can use your "
        "feet your ears and your dollars show up be that receptive and engaged audience that is so necessary for a music "
        "city to thrive pay a cover charge buy a record discover new music and please take your friends two you can use "
        "your voice buy in to the assets based narrative talk about and celebrate what your city has and three you can "
        "use your vote seek out leadership that doesn't just pay lip service to your city's music but recognizes its "
        "power and is prepared to put a strategy in place to elevate it grow it and build collaboration no there really "
        "is no telling what city could be defined by a certain scene or a certain song in the next decade but as much "
        "as we absolutely cannot predict that what we absolutely can predict is what happens when we treat music as "
        "necessary and we work to build a music city and that is a place where i want to live thank you"
    ],
    max_seq_length=128,
    step=8,
    margin=16,
    batch_size=32,
)
print(inference_results[0])

## Training Script

If you have NeMo installed locally, you can also train the model with [nlp/token_classification/punctuation_capitalization_train.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/punctuation_capitalization_train.py).

To run training script, use:

`python punctuation_and_capitalization_train.py model.dataset.data_dir=PATH_TO_DATA_DIR`

Set NUM_SAMPLES=-1 and consider including other datasets to improve the performance of the model.

# Finetuning model with your data

When we were training the model from scratch, the datasets were prepared for training during the model initialization. When we are using a pretrained Punctuation and Capitalization model, before training, we need to setup training and evaluation data.

In [ ]:
# let's reload our pretrained model
pretrained_model = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained('punctuation_en_distilbert')

# setup train and validation Pytorch DataLoaders
pretrained_model.update_data_dir(DATA_DIR)
pretrained_model.setup_training_data()
pretrained_model.setup_validation_data()

# and now we can create a PyTorch Lightning trainer and call `fit` again
# for this tutorial we are setting fast_dev_run to True, and the trainer will run 1 training batch and 1 validation batch
# for actual model training, disable the flag
fast_dev_run = True
trainer = pl.Trainer(gpus=1, fast_dev_run=fast_dev_run)
trainer.fit(pretrained_model)